In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Import extra libraries: matplotlib, scipy, seaborn

In [ ]:
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns

#pd.options.display.max_rows = 1000
#pd.options.display.max_columns = 20

Dataset directory

In [ ]:
dfinputdir = '/kaggle/input/iowa-liquor-mining/liquor.csv'

Preprocessing: 'Store Location'. Calculate % of rows that contain empty fields (is null). Nearly 10% of values in this attribute are missing from a sample size of 1 million.

In [ ]:
colnan = ['Store Location']
dfnan = pd.read_csv(dfinputdir,usecols=colnan,nrows=1000000)
sumnan = dfnan.isnull().sum()
print((sumnan/1000000)*100)

Read dataset into pandas dataframe. This is a subset of the dataset (1 million rows) with 5 numerical attributes: Sale, Pack, Bottles Sold, Bottle Cost, and Bottle Volume.

In [ ]:
cols = ['Sale (Dollars)','Pack','Bottles Sold','State Bottle Cost','State Bottle Retail','Bottle Volume (ml)','Volume Sold (Liters)']
df = pd.read_csv(dfinputdir,usecols=cols,nrows=1000000)

Proprocessing step #1 - view info of dataframe.

In [ ]:
df.info()

Pandas interprets pack, bottle volume, and bottles sold as integer datatypes. Bottle cost and sale are float datatypes.
Outlier analysis focuses on the Sale attribute.

In [ ]:
print(df['Sale (Dollars)'].describe())
#plt.figure(figsize=(9, 8))


The maximum value in Sale is more than 400 standard deviations away from the mean. Conventionally, outliers are detected at >=3 standard deviations from the mean (99.7% of data lies within 3 std). This is appears to be an individual, global outlier.

In [ ]:
def grubbs_test(x):
    n = len(x)
    mean_x = np.mean(x)
    sd_x = np.std(x)
    numerator = max(abs(x-mean_x))
    g_calculated = numerator/sd_x
    print("Grubbs Calculated Value:",g_calculated)
    t_value = stats.t.ppf(1 - 0.05 / (2 * n), n - 2)
    g_critical = ((n - 1) * np.sqrt(np.square(t_value))) / (np.sqrt(n) * np.sqrt(n - 2 + np.square(t_value)))
    print("Grubbs Critical Value:",g_critical)
    if g_critical > g_calculated:
        print("The calculated value is less than the critical value: by Grubbs test there are no outliers\n")
    else:
        print("The calculated value is greater than the critical value: by Grubbs test there is an outlier\n")

In [ ]:
grubbs_test(df['Sale (Dollars)'].to_numpy())

In [ ]:
grubbs_test(df['Pack'].to_numpy())

In [ ]:
grubbs_test(df['Bottles Sold'].to_numpy())

In [ ]:
grubbs_test(df['State Bottle Cost'].to_numpy())

In [ ]:
grubbs_test(df['Bottle Volume (ml)'].to_numpy())

In [ ]:
grubbs_test(df['State Bottle Retail'].to_numpy())

In [ ]:
grubbs_test(df['Volume Sold (Liters)'].to_numpy())

In [ ]:
#Z-score based outlier detection
z_out=[]
def Zscore_outlier(df):
    m = np.mean(df)
    sd = np.std(df)
    for i in df: 
        z = (i-m)/sd
        if np.abs(z) > 3: 
            z_out.append(i)
    print("Outliers:",len(z_out))
Zscore_outlier(df['Sale (Dollars)'])
df_zo = pd.DataFrame (z_out, columns = ['outliers'])
#print(df_zo['outliers'].describe())

df1 = df[df['Sale (Dollars)'].isin(z_out) == False]
print(df1['Sale (Dollars)'].describe())

In [ ]:
#IQR based outlier detection
iqr_out=[]
def iqr_outliers(df):
    q1 = df.quantile(0.25)
    q3 = df.quantile(0.75)
    iqr = q3-q1
    Lower_tail = q1 - 1.5 * iqr
    Upper_tail = q3 + 1.5 * iqr
    for i in df:
        if i > Upper_tail or i < Lower_tail:
            iqr_out.append(i)
    print("Outliers:",len(iqr_out))
iqr_outliers(df['Sale (Dollars)'])
df_iqr = pd.DataFrame (iqr_out, columns = ['outliers'])
#print(df_iqr['outliers'].describe())

df2 = df[df['Sale (Dollars)'].isin(iqr_out) == False]
print(df2['Sale (Dollars)'].describe())

In [ ]:
#Winsorization based outlier detection
w_out=[]
def Winsorization_outliers(df):
    q1 = np.percentile(df , 1)
    q3 = np.percentile(df , 99)
    for i in df:
        if i > q3 or i < q1:
            w_out.append(i)
    print("Outliers:",len(w_out))
Winsorization_outliers(df['Sale (Dollars)'])

df3 = df[df['Sale (Dollars)'].isin(w_out) == False]
print(df3['Sale (Dollars)'].describe())

In [ ]:
plt.title("Raw Data (Outliers Included)")
sns.boxplot(df['Sale (Dollars)']);

In [ ]:
plt.title("Z-Score Based Outlier Detection (6,129 Values Removed)")
sns.boxplot(df1['Sale (Dollars)']);

In [ ]:
plt.title("IQR Based Outlier Detection (66,710 Values Removed)")
sns.boxplot(df2['Sale (Dollars)']);

In [ ]:
plt.title("Winsorization Based Outlier Detection (19,480 Values Removed)")
sns.boxplot(df3['Sale (Dollars)']);

In [ ]:
plt.title("Raw Data (Outliers Included)")
sns.distplot(df['Sale (Dollars)'], color='g', bins=100, hist_kws={'alpha': 0.4});

In [ ]:
plt.title("Z-Score Based Outlier Detection (6,129 Values Removed)")
sns.distplot(df1['Sale (Dollars)'], color='g', bins=100, hist_kws={'alpha': 0.4});

In [ ]:
plt.title("IQR Based Outlier Detection (66,710 Values Removed)")
sns.distplot(df2['Sale (Dollars)'], color='g', bins=100, hist_kws={'alpha': 0.4});

In [ ]:
plt.title("Winsorization Based Outlier Detection (19,480 Values Removed)")
sns.distplot(df3['Sale (Dollars)'], color='g', bins=100, hist_kws={'alpha': 0.4});

In [ ]:
def pairplot(x, y, **kwargs):
    ax = plt.gca()
    ts = pd.DataFrame({'time': x, 'val': y})
    ts = ts.groupby('time').mean()
    ts.plot(ax=ax)
    plt.xticks(rotation=90)
    
f = pd.melt(df, id_vars=['Sale (Dollars)'])
g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False, size=5)
g = g.map(pairplot, "value", "Sale (Dollars)")

In [ ]:
f = pd.melt(df2, id_vars=['Sale (Dollars)'])
g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False, size=5)
g = g.map(pairplot, "value", "Sale (Dollars)")

References

https://www.kaggle.com/nareshbhat/outlier-the-silent-killer

https://www.kaggle.com/dgawlik/house-prices-eda/notebook

https://www.kaggle.com/ekami66/detailed-exploratory-data-analysis-with-python

https://www.kaggle.com/rpsuraj/outlier-detection-techniques-simplified